In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

In [ ]:
col = ['fLenghth', 'fWidth', 'fSize', 'fConc', 'fConc1', 'fAsym', 'fM3Long', 'fM3Trans', 'fAlpha', 'fDist', 'class']

df = pd.read_csv("magic04.data", names=col)
df["class"] = (df["class"] == "g").astype(int)
df.head()

# for labels in col[:-1]:
#   plt.hist(df[df["class"] == 1][labels], color="blue", label="gamma", alpha=0.7, density=True)
#   plt.hist(df[df["class"] == 0][labels], color="red", label="gamma", alpha=0.7, density=True)
#   plt.title(labels)
#   plt.ylabel("Probability")
#   plt.xlabel(labels)
#   plt.legend()
#   plt.show()

,fLenghth,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,1
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,1
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,1
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,1
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,1


# Data Split


In [ ]:
df.sample(frac=1)
train, valid, test = np.split(df.sample(frac=1), [int(0.6*len(df)), int(0.8*len(df))])

In [ ]:
def scale_dataset(dataframe, oversample=False):
  X = dataframe[dataframe.columns[:-1]].values
  y = dataframe[dataframe.columns[-1]].values

  scaler = StandardScaler()
  X = scaler.fit_transform(X)

  if oversample:
    ros = RandomOverSampler()
    X,y = ros.fit_resample(X,y)

  data = np.hstack((X, np.reshape(y, (-1, 1))))
  return data, X, y

In [ ]:
train, X_train, y_train = scale_dataset(train, oversample=True)
valid, X_valid, y_valid = scale_dataset(valid, oversample=False)
test, X_test, y_test = scale_dataset(test, oversample=False)

# KNN


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)

KNeighborsClassifier()

In [ ]:
y_pred = knn_model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.73      0.74      1350
           1       0.85      0.86      0.86      2454

    accuracy                           0.82      3804
   macro avg       0.80      0.80      0.80      3804
weighted avg       0.82      0.82      0.82      3804



# NAIVE BAYES

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
nb_model = GaussianNB()
nb_model = nb_model.fit(X_train, y_train)

In [ ]:
y_pred = nb_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.66      0.41      0.50      1350
           1       0.73      0.89      0.80      2454

    accuracy                           0.72      3804
   macro avg       0.70      0.65      0.65      3804
weighted avg       0.71      0.72      0.70      3804



# Support Vector Machine

In [ ]:
from sklearn.svm import SVC

In [ ]:
svm_model = SVC()
svm_model = svm_model.fit(X_train, y_train)

In [ ]:
y_pred = svm_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.79      0.80      1350
           1       0.88      0.91      0.90      2454

    accuracy                           0.86      3804
   macro avg       0.85      0.85      0.85      3804
weighted avg       0.86      0.86      0.86      3804



# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lg_model = LogisticRegression()
lg_model = lg_model.fit(X_train, y_train)

In [ ]:
y_pred = lg_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.69      0.72      0.71      1350
           1       0.84      0.82      0.83      2454

    accuracy                           0.79      3804
   macro avg       0.77      0.77      0.77      3804
weighted avg       0.79      0.79      0.79      3804



# Neural Network

In [ ]:
import tensorflow as tf

In [ ]:
def plot_history(history):
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
  ax1.plot(history.history['loss'], label='loss')
  ax1.plot(history.history['val_loss'], label='val_loss')
  ax1.set_xlabel('Epoch')
  ax1.set_ylabel('Binary crossentropy')
  ax1.grid(True)

  ax2.plot(history.history['accuracy'], label='accuracy')
  ax2.plot(history.history['val_accuracy'], label='val_accuracy')
  ax2.set_xlabel('Epoch')
  ax2.set_ylabel('Accuracy')
  ax2.grid(True)

  plt.show()

In [ ]:
def train_model(X_train, y_train, num_nodes, dropout_prob, lr, batch_size, epochs):
  nn_model = tf.keras.Sequential([
      tf.keras.layers.Dense(num_nodes, activation='relu', input_shape=(10,)),
      tf.keras.layers.Dropout(dropout_prob),
      tf.keras.layers.Dense(num_nodes, activation='relu'),
      tf.keras.layers.Dropout(dropout_prob),
      tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  nn_model.compile(optimizer=tf.keras.optimizers.Adam(lr), loss='binary_crossentropy',
                  metrics=['accuracy'])
  history = nn_model.fit(
    X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0
  )

  return nn_model, history

In [ ]:
least_val_loss = float('inf')
least_loss_model = None
best_model_nodes = int(0)
best_model_dropout = int(0)
best_model_lr = int(0)
best_model_batch_size = int(0)
epochs=100
for num_nodes in [16, 32, 64]:
  for dropout_prob in[0, 0.2]:
    for lr in [0.01, 0.005, 0.001]:
      for batch_size in [32, 64, 128]:
        print(f"{num_nodes} nodes, dropout {dropout_prob}, lr {lr}, batch size {batch_size}")
        model, history = train_model(X_train, y_train, num_nodes, dropout_prob, lr, batch_size, epochs)
        plot_history(history)
        val_loss = model.evaluate(X_valid, y_valid)[0]
        if val_loss < least_val_loss:
          least_val_loss = val_loss
          least_loss_model = model
          best_model_nodes = num_nodes
          best_model_dropout = dropout_prob
          best_model_lr = lr
          best_model_batch_size = batch_size

16 nodes, dropout 0, lr 0.01, batch size 32
119/119 [==============================] - 0s 2ms/step - loss: 0.3359 - accuracy: 0.8644
16 nodes, dropout 0, lr 0.01, batch size 64
119/119 [==============================] - 0s 2ms/step - loss: 0.3181 - accuracy: 0.8785
16 nodes, dropout 0, lr 0.01, batch size 128
119/119 [==============================] - 0s 2ms/step - loss: 0.3247 - accuracy: 0.8709
16 nodes, dropout 0, lr 0.005, batch size 32
119/119 [==============================] - 0s 2ms/step - loss: 0.3258 - accuracy: 0.8693
16 nodes, dropout 0, lr 0.005, batch size 64
119/119 [==============================] - 0s 2ms/step - loss: 0.3221 - accuracy: 0.8686
16 nodes, dropout 0, lr 0.005, batch size 128
119/119 [==============================] - 0s 2ms/step - loss: 0.3158 - accuracy: 0.8754
16 nodes, dropout 0, lr 0.001, batch size 32
119/119 [==============================] - 0s 2ms/step - loss: 0.3160 - accuracy: 0.8683
16 nodes, dropout 0, lr 0.001, batch size 64
119/119 [=========

In [ ]:
y_pred = least_loss_model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int).reshape(-1,)
print("best_model_nodes = {}".format(best_model_nodes))
print("best_model_dropout = {}".format(best_model_dropout))
print("best_model_lr = {}".format(best_model_lr))
print("best_model_batch_size = {}".format(best_model_batch_size))

119/119 [==============================] - 0s 3ms/step
best_model_nodes = 64
best_model_dropout = 0.2
best_model_lr = 0.001
best_model_batch_size = 64


In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.74      0.81      1350
           1       0.87      0.95      0.91      2454

    accuracy                           0.87      3804
   macro avg       0.88      0.85      0.86      3804
weighted avg       0.88      0.87      0.87      3804

